In [1]:
from langchain.agents import create_sql_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4o",
)
db = SQLDatabase.from_uri("sqlite:///movies.sqlite")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

agent.invoke(
    "Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue."
)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`


directors, movies
Invoking: `sql_db_schema` with `movies, directors`



CREATE TABLE directors (
	name TEXT, 
	id INTEGER, 
	gender INTEGER, 
	uid INTEGER, 
	department TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from directors table:
name	id	gender	uid	department
James Cameron	4762	2	2710	Directing
Gore Verbinski	4763	2	1704	Directing
Sam Mendes	4764	2	39	Directing
*/


CREATE TABLE movies (
	id INTEGER, 
	original_title VARCHAR, 
	budget INTEGER, 
	popularity INTEGER, 
	release_date TEXT, 
	revenue INTEGER, 
	title TEXT, 
	vote_average REAL, 
	vote_count INTEGER, 
	overview TEXT, 
	tagline TEXT, 
	uid INTEGER, 
	director_id INTEGER DEFAULT 0 NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from movies table:
id	original_title	budget	popularity	release_date	revenue	title	vote_average	vote_count	overview	tagline	uid	director_id
43597	Avatar	237000000	150	2009-12-10	2787965087	Avatar	7.2	11800	In the 22nd century, a 

{'input': 'Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue.',
 'output': 'Here are some movies that have the highest votes but the lowest budgets, along with their directors and gross revenue:\n\n1. **Inception**\n   - Director: Christopher Nolan\n   - Votes: 13,752\n   - Budget: $160,000,000\n   - Revenue: $825,532,764\n\n2. **The Dark Knight**\n   - Director: Christopher Nolan\n   - Votes: 12,002\n   - Budget: $185,000,000\n   - Revenue: $1,004,558,444\n\n3. **Avatar**\n   - Director: James Cameron\n   - Votes: 11,800\n   - Budget: $237,000,000\n   - Revenue: $2,787,965,087\n\n4. **The Avengers**\n   - Director: Joss Whedon\n   - Votes: 11,776\n   - Budget: $220,000,000\n   - Revenue: $1,519,557,910\n\n5. **Deadpool**\n   - Director: Tim Miller\n   - Votes: 10,995\n   - Budget: $58,000,000\n   - Revenue: $783,112,979\n\n6. **Interstellar**\n   - Director: Christopher Nolan\n   - Votes: 10,8